<a href="https://colab.research.google.com/github/kanavgoyal781/Gen_Ai/blob/Step-1-Choose_a_Model/Fine_Tuning_LLMs_with_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
# Install core dependencies for LoRA + supervised fine-tuning (SFTTrainer setup)
%pip install -q --upgrade \
  transformers \
  datasets \
  peft==0.8.2 \
  accelerate \
  bitsandbytes \
  trl==0.8.6 \
  wandb \
  evaluate \
  rouge_score


In [ ]:
from huggingface_hub import login
login()  # Then paste your updated access token

In [ ]:
!pwd

In [ ]:
!pip install nbstripout
!nbstripout /content/Fine_Tuning_LLMs_with_QLoRA.ipynb

Step1

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import torch
from transformers import BitsAndBytesConfig

### 📦 Library Imports
- **AutoModelForCausalLM**: From Hugging Face’s Transformers; loads a pre‑trained causal language model (e.g., Mistral, LLaMA).
- **AutoTokenizer**: Loads the matching tokenizer for the model (handles text → tokens).
- **LoraConfig**: Defines LoRA adapter parameters (e.g., rank, alpha, dropout).
- **get_peft_model**: Applies the LoRA adapters to the base model.
- **TaskType**: Specifies what kind of task LoRA is applied for (e.g., Causal LM).
- **torch**: Core PyTorch library for tensors and GPU computation.
- **BitsAndBytesConfig**: Configures 4‑bit quantization via `bitsandbytes` (reduces memory).

In [13]:
model_name = "mistralai/Mistral-7B-v0.1"

### 🎯 Model Selection
- **model_name**: The Hugging Face ID of the model we want to load.
- Here we selected **Mistral‑7B**, a 7‑billion parameter model, fully compatible with LoRA.

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]